# REDES NEURAIS ARTIFICIAIS

# PARTE 1: Classificação Binária - Diagnóstico de Câncer de Mama

Base de dados: Breast Cancer Wisconsin (Diagnostic) -
https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic

In [27]:
#bilbiotecas keras e tensorflow
%pip install scikeras==0.13.0

Note: you may need to restart the kernel to use updated packages.


In [28]:
%pip install tensorflow==2.16.1 scikit-learn==1.5.0

In [29]:
# após instalar as bibliotecas, é necessário reiniciar o notebook
# Para isso, rode esta célula sozinha antes de prosseguir.
exit()

In [1]:
import pandas as pd
import tensorflow as tf
import sklearn
import scikeras
from sklearn.model_selection import train_test_split

In [2]:
pd.__version__, tf.__version__, sklearn.__version__, scikeras.__version__

('2.3.3', '2.16.1', '1.5.0', '0.13.0')

### Carregar os dados

1) Carregue a base de dados, faça a divisição de treino e teste (para isso, utilize a função train_test_split do sklearn), como o tamanho da base de teste de 0.25.

In [3]:
dados = pd.read_csv("dados_breast.csv")
rotulos = pd.read_csv("rotulos_breast.csv")

X = dados / 255.0
y = rotulos

X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X, y, test_size=0.25, random_state=42)

In [4]:
X_treinamento.shape, y_treinamento.shape

((426, 30), (426, 1))

In [6]:
X_teste.shape, y_teste.shape

((143, 30), (143, 1))

### Estrutura da Rede Neural Artificial e Teste

In [7]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow.keras import backend as k
from tensorflow.keras.layers import Dense
from tensorflow.keras.initializers import RandomUniform
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy

2) Crie a RNA com as seguintes configurações:

a) uma camada de entrada de com 30 neurônios;

b) uma camada oculta densa com 16 neurônios. Porque a camada oculta tem essa quantidade de neurônios?

c) adicione na camada oculta a função de ativação relu e inicialize os pesos de utilize o Random uniform initializer (https://www.tensorflow.org/api_docs/python/tf/keras/initializers);

d) adicione a camada de saíde com a função de ativação sigmoid. Porque foi utilizada esta função para a saída da RNA?

3) Porque utilizamos a classe chamada Sequential para a RNA?

In [8]:
'''
b) Geralmente escolhemos um número que seja suficiente para a rede aprender padrões complexos, mas não tão grande que torne o treinamento lento ou cause overfitting
d) A função Sigmoid é usada porque ela transforma qualquer valor que venha dos neurônios anteriores em um número entre 0 e 1
3) Usamos a classe Sequential porque ela é a maneira mais simples e direta de construir modelos no keras, onde as camadas são empilhadas em linha reta, uma após a outra
'''
rede_neural = Sequential()

# Camada de entrada — 30 neurônios
rede_neural.add(Dense(
    units=30,
    activation='relu',
    kernel_initializer=RandomUniform(minval=-0.05, maxval=0.05),
    input_shape=(30,)
))

# Camada oculta — 16 neurônios
rede_neural.add(Dense(
    units=16,
    activation='relu',
    kernel_initializer=RandomUniform(minval=-0.05, maxval=0.05)
))

# Camada de saída — 1 neurônio (classificação binária)
rede_neural.add(Dense(
    units=1,
    activation='sigmoid'
))

c:\Users\deant\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4) A partir da RNA gerada, explique o que são os valores apresentados na tabela da rede_neural.summary()

In [9]:
'''
4)
Layer: Mostra o nome e o tipo da camada
Output Shape: Mostra a dimensão dos dados que saem dessa camada
Param #: É a quantidade de ajustes ou botões internos que a rede vai aprender durante o treinamento
'''
rede_neural.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 30)             │           930 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,443 (5.64 KB)

 Trainable params: 1,443 (5.64 KB)

 Non-trainable params: 0 (0.00 B)

5) Adicione um otimizador Adam (https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) e especifique a classe loss binário - binary crossentropy (https://www.tensorflow.org/api_docs/python/tf/keras/losses) e a classe metrics para utilizar a métrica de avaliação de acurácia binária (https://www.tensorflow.org/api_docs/python/tf/keras/metrics)

6) Para que servem os otimizadores? Como o otimizador Adam funciona?

7) Depois de estruturados os parâmtros da RNA, utilize a função .fit para fazer o treinamento da rede. Como foi utilizado o otimizador Adam, e ele é baseado na descida do gradiente estocástica, é possível definir a quantidade de registros que serão enviados para a RNA, isto é, em cada batch serão utilizados 10 registros. Quantos batches serão utilizados ao total?

8) Por fim, defina o número de épocas em que ocorre o treinamento igual a 100.

In [10]:
'''
6) os otimizadores são o motor de aprendizado da rede neural
O otimizador usa o cálculo do rgadiente (a inclinação do erro) para determinar
a direção e a intensidade com que os pesos internos da rede devem ser ajustados 
para que o erro diminua
ou seja, ele garante que a rede se mova em direção ao ponto onde o erro é o menor possível
7)o tamanho do lote é 10 e os dados de treinamento são 45.000 registros, como vamos enviar os
dados em pacotes de 10 registros por lote (batch), a rede utilizará 4.500 lotes por época
a cada vez que a rede completa uma época, ela executa 4.500 passos de ajuste para aprender
com todos os dados
o treinamento total será de 4.500 lotes x 100 épocas = 450.000 lotes
'''

rede_neural.compile(
    optimizer=Adam(),
    loss=BinaryCrossentropy(),
    metrics=[BinaryAccuracy()]
)


historia = rede_neural.fit(
    X_treinamento, 
    y_treinamento, 
    
    batch_size=10, 
    epochs=100
)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - binary_accuracy: 0.6854 - loss: 0.6479
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7230 - loss: 0.5534
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.7653 - loss: 0.4908
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8427 - loss: 0.4335
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8638 - loss: 0.3854
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8756 - loss: 0.3351
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8803 - loss: 0.3038
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8944 - loss: 0.2725
Epoch 9/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8967 - loss: 0.2609
Epoch 10/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8967 - loss: 0.2503
Epoch 11/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8850 -

9) Crie uma variável chamada previsoes para realizar a previsão dos dados de teste (X_teste) O resultado da rede deve ser um valor entre 0 e 1. Porque isso acontece?

In [11]:
'''
9)isso acontece por causa da ativação utilizada na camada de saída da rede neural, a função sigmoid
'''
previsoes = rede_neural.predict(X_teste)
previsoes

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


array([[9.1258520e-01],
       [7.6607867e-05],
       [1.6783349e-02],
       [9.9722755e-01],
       [9.9928319e-01],
       [2.0700710e-07],
       [2.8119905e-06],
       [2.1864544e-01],
       [9.8420084e-01],
       [9.6413231e-01],
       [9.7067314e-01],
       [8.9699787e-04],
       [9.5074105e-01],
       [6.1283892e-01],
       [1.0000000e+00],
       [1.1998951e-01],
       [9.8232508e-01],
       [9.9680942e-01],
       [1.0000000e+00],
       [1.6072014e-04],
       [7.9093939e-03],
       [9.9531817e-01],
       [4.2397834e-05],
       [9.9920487e-01],
       [9.9809861e-01],
       [9.9894238e-01],
       [9.9350017e-01],
       [9.9960899e-01],
       [9.9460334e-01],
       [4.7696478e-05],
       [9.9950236e-01],
       [9.9767536e-01],
       [1.0000000e+00],
       [8.5400099e-01],
       [9.9952948e-01],
       [9.9797308e-01],
       [1.1079733e-02],
       [9.9595881e-01],
       [1.4395667e-03],
       [8.3208328e-01],
       [9.9761522e-01],
       [1.590474

10) Converta o resultado que está em probabilidade para valores binários com um limiar (treshold) de 0.5.


In [12]:
previsoes = (previsoes >= 0.5).astype(int)

In [13]:
previsoes

array([[1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
    

9) Explique o resultado da RNA.

In [14]:
'''
9) binary accuracy de 0.92 significa que o modelo acertou em 92& das vezes. 
Loss de 0.145 sugere que as saídas são consistentes e próximas das verdadeiras
'''
rede_neural.evaluate(X_teste, y_teste)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - binary_accuracy: 0.9371 - loss: 0.1367  


[0.13667696714401245, 0.9370629191398621]

## Camadas e Otimização da RNA

10) Adicione mais uma camada para oculta densa com 16 neurônio com a função de ativação relu e inicialize os pesos de utilize o Random uniform initializer. Qual o total de parâmetros da RNA agora?

In [ ]:
'''
10) o total de parametros é de 1,715
'''
rede_neural = Sequential()

# Camada de entrada — 30 neurônios
rede_neural.add(Dense(
    units=30,
    activation='relu',
    kernel_initializer=RandomUniform(minval=-0.05, maxval=0.05),
    input_shape=(30,)
))

# Camada oculta — 16 neurônios
rede_neural.add(Dense(
    units=16,
    activation='relu',
    kernel_initializer=RandomUniform(minval=-0.05, maxval=0.05)
))

# Camada oculta — 16 neurônios
rede_neural.add(Dense(
    units=16,
    activation='relu',
    kernel_initializer=RandomUniform(minval=-0.05, maxval=0.05)
))

# Camada de saída — 1 neurônio (classificação binária)
rede_neural.add(Dense(
    units=1,
    activation='sigmoid'
))


rede_neural.summary()

c:\Users\deant\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 30)             │           930 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 16)             │           496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,715 (6.70 KB)

 Trainable params: 1,715 (6.70 KB)

 Non-trainable params: 0 (0.00 B)

11) Abaixo são adicionados os parâmetros do otimizador, que são a taxa de aprendizado e o clipvalue. O que eles fazem?

In [20]:
'''
Taxa de aprendizado é o tamanho dos passos que o otimiazador dá ao aprender. Uma taxa menor é mais lenta mas mais precisa.
Uma taxa maior torna o aprendizado mais rápido, mas pode ser impreciso
Clipvalue faz o gradient clipping, limitando o valor máximo dos gradientes a 0.5. Isso ajuda o treinamento a ser mais estável.
'''
otimizador = tf.keras.optimizers.Adam(learning_rate = 0.001, clipvalue = 0.5)
rede_neural.compile(optimizer = otimizador, loss = 'binary_crossentropy', metrics = ['binary_accuracy'])

12) Teste novamente a RNA. Aumentar a quantidade de camadas melhorou ou piorou os resultados? Explique o que aconteceu com a RNA e porque.

In [21]:
rede_neural.fit(X_treinamento, y_treinamento, batch_size = 10, epochs = 100)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - binary_accuracy: 0.6291 - loss: 0.6755
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.6948 - loss: 0.5940
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8216 - loss: 0.5194
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8380 - loss: 0.4233
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8709 - loss: 0.3447
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8850 - loss: 0.2869
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - binary_accuracy: 0.8944 - loss: 0.2537
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8803 - loss: 0.2536
Epoch 9/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.9038 - loss: 0.2339
Epoch 10/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.9178 - loss: 0.2166
Epoch 11/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.9061 -

In [22]:
previsoes = rede_neural.predict(X_teste)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


In [ ]:
'''
12) A acurácia aumentou de 0.92 para 0.94 e o loss de 0.14 para 0.16
Ou seja, o modelo acecrtou mais vezes, mas tem menos confiança nesses acertos.
Isso ocorre pois uma rede maior tem mais parâmetros, e durante o treinamento ela pode produzir probabilidades menos extremas (mais suaves)
a função de perda sensível ao quão confiantes as previsões são mesmo que acerte mais classes, ela pode estar dando probabilidades mais moderadas
'''
rede_neural.evaluate(X_teste, y_teste)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - binary_accuracy: 0.9441 - loss: 0.1657 


[0.1657053679227829, 0.9440559148788452]

## K-Fold Cross Validation

13) Assista o vídeo https://youtu.be/RczbeFs_WbQ?si=f1Yu4LZaGDfZZpbZ para compreender melhor sobre como funciona o k-fold cross validation. Explique como esta rede foi configurada e como é possível chegar no resultado dela. O que é necessário fazer?


14) Calcule também o Desvio Padrão dos resultados para avaliar o modelo. O que é possível concluir com esse resultado?



In [24]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as k

In [25]:
def criar_rede():
  k.clear_session()
  rede_neural = Sequential([
      tf.keras.layers.InputLayer(shape=(30,)),
      tf.keras.layers.Dense(units=16, activation='relu', kernel_initializer='random_uniform'),
      tf.keras.layers.Dense(units=16, activation='relu', kernel_initializer='random_uniform'),
      tf.keras.layers.Dense(units=1, activation = 'sigmoid')])
  otimizador = tf.keras.optimizers.Adam(learning_rate = 0.001, clipvalue = 0.5)
  rede_neural.compile(optimizer = otimizador, loss = 'binary_crossentropy', metrics = ['binary_accuracy'])
  return rede_neural

In [26]:
rede_neural = KerasClassifier(model = criar_rede, epochs = 100, batch_size = 10)

In [27]:
resultados = cross_val_score(estimator = rede_neural, X = X, y = y, cv = 10, scoring = 'accuracy')


Epoch 1/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - binary_accuracy: 0.6504 - loss: 0.6565
Epoch 2/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.7578 - loss: 0.5661
Epoch 3/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8340 - loss: 0.4851
Epoch 4/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8320 - loss: 0.4216
Epoch 5/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8926 - loss: 0.3620
Epoch 6/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.9023 - loss: 0.3142
Epoch 7/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.9023 - loss: 0.2831
Epoch 8/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - binary_accuracy: 0.8984 - loss: 0.2667
Epoch 9/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.9062 - loss: 0.2482
Epoch 10/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.9082 - loss: 0.2395
Epoch 11/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8926 

In [28]:
resultados

array([0.89473684, 0.89473684, 0.9122807 , 0.94736842, 0.94736842,
       0.89473684, 0.94736842, 0.94736842, 0.89473684, 0.94642857])

In [34]:
media = resultados.mean()

desvio_padrao = resultados.std()
print(f'media = {media}, desvio padrão = {desvio_padrao}')

media = 0.9227130325814535, desvio padrão = 0.02496694207586037


Respostas etapas 13 e 14:

13) Para avaliar o modelo de forma mais robusta e evitar dependência de um único particionamento aleatório, foi aplicado o método de validação cruzada k-fold com cv = 10. Nesse método O dataset é dividido em 10 partes, a rede é treinada 10 vezes (sempre usando 9 para treino e 1 para teste), a acurácia final é a média das 10 execuções

14) O devio padrão foi de 0.02 isso significa que a variação entre os diferentes folds foi baixa. Ou seja, o modelo é estável. 

## Overfitting e Dropout

14) Como vimos, o modelo anterior sofreu overfitting. Para reduzir esse problema, aplicamos a técnica de regularização conhecida como dropout (artigo oficial: https://www.cs.toronto.edu/~rsalakhu/papers/srivastava14a.pdf e explicação em português: https://www.deeplearningbook.com.br/como-funciona-o-dropout/ ). Aplique o dropout de 20% na primeira e segunda camada oculta. O que acontece com os resultados? E o Desvio Padrão?

In [37]:
def criar_rede():
  k.clear_session()
  rede_neural = Sequential([
      tf.keras.layers.InputLayer(shape=(30,)),
      tf.keras.layers.Dense(units=16, activation='relu', kernel_initializer='random_uniform'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(units=16, activation='relu', kernel_initializer='random_uniform'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(units=1, activation = 'sigmoid')])
  otimizador = tf.keras.optimizers.Adam(learning_rate = 0.001, clipvalue = 0.5)
  rede_neural.compile(optimizer = otimizador, loss = 'binary_crossentropy', metrics = ['binary_accuracy'])
  return rede_neural

rede_neural = KerasClassifier(model = criar_rede, epochs = 100, batch_size = 10)

In [39]:
resultados = cross_val_score(estimator = rede_neural, X = X, y = y, cv = 10, scoring = 'accuracy')

Epoch 1/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - binary_accuracy: 0.6758 - loss: 0.6771
Epoch 2/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7383 - loss: 0.5980
Epoch 3/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.7520 - loss: 0.5288
Epoch 4/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.7793 - loss: 0.4824
Epoch 5/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8125 - loss: 0.4374
Epoch 6/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - binary_accuracy: 0.8438 - loss: 0.4032
Epoch 7/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8711 - loss: 0.3567
Epoch 8/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - binary_accuracy: 0.8750 - loss: 0.3371
Epoch 9/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8770 - loss: 0.3192
Epoch 10/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - binary_accuracy: 0.8789 - loss: 0.3121
Epoch 11/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - binary_accuracy: 0.8809 -

In [ ]:
'''
O desvio padrão foi reduzido de 0.024 para 0.020
'''

resultados

media = resultados.mean()

desvio_padrao = resultados.std()
print(f'media = {media}, desvio padrão = {desvio_padrao}')

media = 0.924436090225564, desvio padrão = 0.0208038921751705


## Tuning dos Hiperparâmetros (técnicas de ajuste)

Hyperparameter Tuning (Ajuste de Hiperparâmetros) é o processo geral de encontrar os melhores valores para os hiperparâmetros de um modelo de machine learning.
Exemplos de hiperparâmetros:

- Taxa de aprendizado (learning rate)

- Número de árvores em um RandomForest

- Número de camadas ou neurônios em uma rede neural

- Tipo de kernel em um SVM

O tuning pode ser feito por diferentes técnicas, como:

- Grid Search

- Random Search

- Bayesian Optimization, etc.

15) Descreva como a RNA foi configurada para fazer o processo de tuning.

In [42]:
def criar_rede(optimizer, loss, kernel_initializer, activation, neurons):
  k.clear_session()
  rede_neural = Sequential([
      tf.keras.layers.InputLayer(shape=(30,)),
      tf.keras.layers.Dense(units=neurons, activation=activation, kernel_initializer=kernel_initializer),
      tf.keras.layers.Dropout(rate = 0.2),
      tf.keras.layers.Dense(units=neurons, activation=activation, kernel_initializer=kernel_initializer),
      tf.keras.layers.Dropout(rate = 0.2),
      tf.keras.layers.Dense(units=1, activation = 'sigmoid')])
  rede_neural.compile(optimizer = optimizer, loss = loss, metrics = ['binary_accuracy'])
  return rede_neural

In [ ]:
rede_neural = KerasClassifier(model = criar_rede)

In [47]:
parametros = {
    'batch_size': [10, 30],
    'epochs': [50, 100],
    'model__optimizer': ['adam', 'sgd'],
    'model__loss': ['binary_crossentropy', 'hinge'],
    'model__kernel_initializer': ['random_uniform', 'normal'],
    'model__activation': ['relu', 'tanh'],
    'model__neurons': [16, 8]
}

In [53]:
#caso tiver problema com a configuração anterior, testar essa
parametros = {
    'batch_size': [10, 30],
    'epochs': [50],
    'model__optimizer': ['adam'],
    'model__loss': ['binary_crossentropy'],
    'model__kernel_initializer': ['random_uniform', 'normal'],
    'model__activation': ['relu'],
    'model__neurons': [16]
}

In [54]:
parametros

{'batch_size': [10, 30],
 'epochs': [50],
 'model__optimizer': ['adam'],
 'model__loss': ['binary_crossentropy'],
 'model__kernel_initializer': ['random_uniform', 'normal'],
 'model__activation': ['relu'],
 'model__neurons': [16]}

In [55]:
from sklearn.model_selection import GridSearchCV

In [56]:
grid_search = GridSearchCV(estimator = rede_neural, param_grid = parametros,
                          scoring = 'accuracy', cv = 5)

In [57]:
grid_search = grid_search.fit(X, y)

Epoch 1/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - binary_accuracy: 0.6330 - loss: 0.6808
Epoch 2/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.7297 - loss: 0.6247
Epoch 3/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.7714 - loss: 0.5604
Epoch 4/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.7495 - loss: 0.5038
Epoch 5/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - binary_accuracy: 0.7846 - loss: 0.4676
Epoch 6/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8066 - loss: 0.4476
Epoch 7/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - binary_accuracy: 0.8374 - loss: 0.4118
Epoch 8/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8659 - loss: 0.3834
Epoch 9/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8879 - loss: 0.3481
Epoch 10/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8703 - loss: 0.3324
Epoch 11/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8703 - loss: 0.32

In [58]:
print(grid_search)

GridSearchCV(cv=5,
             estimator=KerasClassifier(model=<function criar_rede at 0x000002247ACDA0C0>),
             param_grid={'batch_size': [10, 30], 'epochs': [50],
                         'model__activation': ['relu'],
                         'model__kernel_initializer': ['random_uniform',
                                                       'normal'],
                         'model__loss': ['binary_crossentropy'],
                         'model__neurons': [16], 'model__optimizer': ['adam']},
             scoring='accuracy')


In [59]:
melhores_parametros = grid_search.best_params_
print(melhores_parametros)

{'batch_size': 10, 'epochs': 50, 'model__activation': 'relu', 'model__kernel_initializer': 'normal', 'model__loss': 'binary_crossentropy', 'model__neurons': 16, 'model__optimizer': 'adam'}


In [60]:
melhor_precisao = grid_search.best_score_
print(melhor_precisao)

0.9156963204471354


Resposta 15:
O tuning foi feito via grid search.

16) É possível melhorar ainda mais a curácia da RNA? Como?

Resposta 16:
Usando outras técnicas de tuning (como bayesian otimization e random search), pode ser possível aumentar a acurácia.

# Para salvar o seu modelo de RNA

In [ ]:
classificador.save('classificador_breast.keras')

In [ ]:
classificador_novo = tf.keras.models.load_model('/content/classificador_breast.keras')

In [ ]:
classificador_novo.summary()